In [1]:
# Importing dependencies

import os
import logging
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Dense, GlobalAveragePooling2D, Flatten, BatchNormalization, Activation, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception, DenseNet201


IMG_SIZE = 224, 224
BATCH_SIZE = 64
SEED = 6
AUTO = tf.data.AUTOTUNE
tf.random.set_seed(SEED)

# <span style="color:#e74c3c;"> Reading </span> Data

In [3]:
# Creating main dataframe contains image paths and their classes.

def create_images_list(path):
    full_path = []
    images = sorted(os.listdir(path))
    for i in images:
        full_path.append(os.path.join(path, i))
    return full_path


def create_full_img_class_df(full_path):
    full_images_path = create_images_list(full_path)
    
    data = pd.DataFrame()
    img_rows = []
    label_rows = []
    
    for i in range(len(full_images_path)):
        x = create_images_list(full_images_path[i])
        img_rows.extend(x)
        label_rows.extend([i]*len(x))
    
    data['img'] = img_rows
    data['label'] = label_rows
    
    return data


# look-up table
classes = {0: 'african_elephant', 1: 'alpaca', 2: 'american_bison', 3: 'anteater', 4: 'arctic_fox', 5: 'armadillo', 6: 'baboon', 7: 'badger', 8: 'blue_whale', 9: 'brown_bear', 10: 'camel', 11: 'dolphin', 12: 'giraffe', 13: 'groundhog', 14: 'highland_cattle', 15: 'horse', 16: 'jackal', 17: 'kangaroo', 18: 'koala', 19: 'manatee', 20: 'mongoose', 21: 'mountain_goat', 22: 'opossum', 23: 'orangutan', 24: 'otter', 25: 'polar_bear', 26: 'porcupine', 27: 'red_panda', 28: 'rhinoceros', 29: 'sea_lion', 30: 'seal', 31: 'snow_leopard', 32: 'squirrel', 33: 'sugar_glider', 34: 'tapir', 35: 'vampire_bat', 36: 'vicuna', 37: 'walrus', 38: 'warthog', 39: 'water_buffalo', 40: 'weasel', 41: 'wildebeest', 42: 'wombat', 43: 'yak', 44: 'zebra'}

full_data = create_full_img_class_df('/kaggle/input/mammals-image-classification-dataset-45-animals/mammals')

# <span style="color:#e74c3c;"> Creating </span> Train, Validation, Test Sets

In [4]:
y = full_data.pop('label')
X = full_data

X_data, X_test, y_data, y_test =  train_test_split(X, y, test_size = 0.1, stratify = y, random_state=SEED, shuffle=True)
X_train, X_val, y_train, y_val =  train_test_split(X_data, y_data, test_size = 0.09, stratify = y_data, random_state=SEED, shuffle=True)

print("train shape -> ", X_train.shape[0])
print("val shape -> ", X_val.shape[0])
print("test shape -> ", X_test.shape[0])

train shape ->  11261
val shape ->  1114
test shape ->  1376


In [5]:
# Connecting TPU 
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)

tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

stg = tf.distribute.experimental.TPUStrategy(resolver)

clear_output()

# <span style="color:#e74c3c;"> Creating </span> tf.data Pipeline

In [6]:
# Reading -> Resizing -> Normalization
def img_preprocessing(image, label):
    img = tf.io.read_file(image)
    img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = (IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    
    return img, label

# Data augmentation 
def augmentation(image, label):
    img = tf.image.random_flip_left_right(image, seed = SEED)
    img = tf.image.random_flip_up_down(img, seed = SEED)
    img = tf.image.random_brightness(img, 0.1, seed = SEED)
    img = tf.image.random_contrast(img, 0.2, 0.4, seed = SEED)
    img = tf.image.random_saturation(img, 2, 4, seed = SEED)
    
    return img, label

In [7]:
# Creating dataset loaders and tf.datasets

train_loader = tf.data.Dataset.from_tensor_slices((X_train['img'], y_train))
train_dataset = (train_loader
                 .map(img_preprocessing, num_parallel_calls = AUTO)
                 .map(augmentation, num_parallel_calls = AUTO)
                 .shuffle(BATCH_SIZE*15)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO))


valid_loader = tf.data.Dataset.from_tensor_slices((X_val['img'], y_val))
valid_dataset = (valid_loader
                 .map(img_preprocessing, num_parallel_calls = AUTO)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO))
    
    
test_loader = tf.data.Dataset.from_tensor_slices((X_test['img'], y_test))
test_dataset = (test_loader
                 .map(img_preprocessing, num_parallel_calls = AUTO)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO))

In [25]:
# ama cnn bw ka fashali hena
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),  # Changed input shape
    MaxPooling2D(2, 2),

    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(45, activation='softmax')  # Adjust the number of units to match the number of classes
])


In [26]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])


In [27]:
EPOCHS = 10  # Set the number of epochs

history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    validation_data=valid_dataset)

Epoch 1/10
176/176 [==============================] - 76s 426ms/step - loss: 3.5980 - accuracy: 0.0718 - val_loss: 3.5747 - val_accuracy: 0.0727
Epoch 2/10
176/176 [==============================] - 75s 424ms/step - loss: 3.4730 - accuracy: 0.0956 - val_loss: 3.4687 - val_accuracy: 0.1149
Epoch 3/10
176/176 [==============================] - 75s 423ms/step - loss: 3.4045 - accuracy: 0.1098 - val_loss: 3.6367 - val_accuracy: 0.0898
Epoch 4/10
176/176 [==============================] - 75s 423ms/step - loss: 3.3491 - accuracy: 0.1189 - val_loss: 3.4182 - val_accuracy: 0.1257
Epoch 5/10
176/176 [==============================] - 75s 424ms/step - loss: 3.2911 - accuracy: 0.1352 - val_loss: 3.4212 - val_accuracy: 0.1203
Epoch 6/10
176/176 [==============================] - 75s 424ms/step - loss: 3.2448 - accuracy: 0.1436 - val_loss: 3.6191 - val_accuracy: 0.1095
Epoch 7/10
176/176 [==============================] - 75s 423ms/step - loss: 3.1970 - accuracy: 0.1533 - val_loss: 3.7235 - val_ac

In [28]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy}")


22/22 [==============================] - 3s 134ms/step - loss: 4.4375 - accuracy: 0.0923
Test accuracy: 0.09229651093482971


# <span style="color:#e74c3c;"> Model 1 </span> Xception

In [9]:
hist1 = xception_model.fit(train_dataset, batch_size = BATCH_SIZE, epochs = 10, validation_data = valid_dataset)
xception_model.save('xception_model.h5')

Epoch 1/10


2024-01-02 01:50:24.476252: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AdamW/AssignAddVariableOp.
2024-01-02 01:50:26.923520: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:26.925783: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:26.929554: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:26.931390: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


  2/176 [..............................] - ETA: 17s - loss: 3.8620 - acc: 0.0234 - top_10_cat_acc: 0.2578    

2024-01-02 01:50:58.813424: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:58.813568: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:58.813687: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:58.813778: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:58.813870: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:58.814023: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2024-01-02 01:50:58.814140: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

  4/176 [..............................] - ETA: 17s - loss: 3.8311 - acc: 0.0195 - top_10_cat_acc: 0.2773

...

176/176 [==============================] - 16s 88ms/step - loss: 0.6456 - acc: 0.8414 - top_10_cat_acc: 0.9797 - val_loss: 0.3250 - val_acc: 0.9165 - val_top_10_cat_acc: 0.9910
Epoch 3/10
  2/176 [..............................] - ETA: 13s - loss: 0.5474 - acc: 0.8672 - top_10_cat_acc: 0.9844 

2024-01-02 01:51:46.710066: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


176/176 [==============================] - ETA: 0s - loss: 0.3596 - acc: 0.9082 - top_10_cat_acc: 0.9926

2024-01-02 01:52:01.919444: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


176/176 [==============================] - 16s 89ms/step - loss: 0.3596 - acc: 0.9082 - top_10_cat_acc: 0.9926 - val_loss: 0.3683 - val_acc: 0.9048 - val_top_10_cat_acc: 0.9883
Epoch 4/10
176/176 [==============================] - 16s 88ms/step - loss: 0.2255 - acc: 0.9411 - top_10_cat_acc: 0.9970 - val_loss: 0.3462 - val_acc: 0.9183 - val_top_10_cat_acc: 0.9910
Epoch 5/10
176/176 [==============================] - 16s 88ms/step - loss: 0.1626 - acc: 0.9600 - top_10_cat_acc: 0.9978 - val_loss: 0.3375 - val_acc: 0.9120 - val_top_10_cat_acc: 0.9919
Epoch 6/10
176/176 [==============================] - 16s 86ms/step - loss: 0.1280 - acc: 0.9669 - top_10_cat_acc: 0.9985 - val_loss: 0.3327 - val_acc: 0.9138 - val_top_10_cat_acc: 0.9928
Epoch 7/10
176/176 [==============================] - 16s 87ms/step - loss: 0.0986 - acc: 0.9740 - top_10_cat_acc: 0.9993 - val_loss: 0.3895 - val_acc: 0.9031 - val_top_10_cat_acc: 0.9937
Epoch 8/10
176/176 [==============================] - 16s 91ms/step - l

# <span style="color:#e74c3c;"> Model Test  </span> Results, Predictions

In [16]:
with stg.scope():
    model1_eval = xception_model.evaluate(test_dataset)
    print('Test Loss: {0:.3f}'.format(model1_eval[0]))
    print('Test Accuracy: {0:.3f} %'.format(model1_eval[1]*100))
    print('Test 10 Classes Accuracy: {0:.3f} %'.format(model1_eval[2]*100))

22/22 [==============================] - 2s 30ms/step - loss: 0.3553 - acc: 0.9084 - top_10_cat_acc: 0.9927
Test Loss: 0.355
Test Accuracy: 90.843 %
Test 10 Classes Accuracy: 99.273 %


In [ ]:
# Predictions

with stg.scope():
    test_take =  test_dataset.take(-1)
    test_take_ = list(test_take)
    pred1 = xception_model.predict(test_take)
    pred1 = np.argmax(pred1, axis = 1)
    
    
# Predictions and scores
clf1= classification_report(y_test, pred1, target_names = list(classes.values()),  output_dict = True)
clf1_ = classification_report(y_test, pred1, target_names = list(classes.values()))

f1 = f1_score(y_test, pred1, average = 'weighted')
acc = accuracy_score(y_test, pred1)

print('Weighted F1 Score : {0:.3f}'.format(f1))
print('Accuracy Score : {0:.3f} %'.format(acc*100))
print('--'*30)
print(clf1_)